# DCGAN for selfie images

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import imageio
import warnings
from tqdm import tqdm
from PIL import Image
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from keras import initializers

%matplotlib inline
warnings.simplefilter('ignore')

## Local image preprocessing

In [17]:
selfies = []
DIR = './selfies'

for filename in tqdm(os.listdir(DIR)):
    path = os.path.join(DIR, filename)
    image = Image.open(path)
    image = image.convert('RGB')
    image = image.resize((32, 32), Image.ANTIALIAS)
    selfies.append(np.array(image))
    
np.savez('selfie_avatar.npz', selfies)
print(np.array(selfies).shape)

100%|██████████| 133/133 [00:00<00:00, 153.11it/s]


(133, 64, 64, 3)


In [28]:
''' load image array from disk '''
avatar = np.load('./selfie_avatar_32.npz')
avatar = avatar['arr_0']
print(avatar.shape)

(133, 64, 64, 3)


In [ ]:
plt.figure(figsize=(8,6))
for i in range(20):
    plt.subplot(4, 5, i+1)
    plt.axis('off')
    plt.imshow(avatar[i+6])
    
plt.tight_layout()

In [30]:
# hyperparameters 
init = initializers.RandomNormal(mean=0.0, stddev=0.02)
opt = Adam(lr=0.0002, beta_1=0.5)

## models

In [47]:
''' SHALLOW MODEL @ 32-X-32 '''

img_shape = (32, 32, 3)
z_dim = 100
init = initializers.RandomNormal(mean=0.0, stddev=0.02)
opt = Adam(lr=0.0002, beta_1=0.5)

def build_discriminator(in_shape=img_shape):
    model = Sequential()
    
    model.add(Conv2D(64, (5,5), input_shape=in_shape, kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(256, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(512, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
#     model.summary()
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

def build_generator(latent_dim):
    model = Sequential()
    
    model.add(Dense(512*4*4, input_dim=latent_dim, kernel_initializer=init))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 512)))
    
    model.add(Conv2DTranspose(256, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(64, (5,5), strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same', kernel_initializer=init))
    
#     model.summary()
    
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    
#     model.summary()
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

## helper methods

In [36]:
def load_real_samples():
    selfie_data = np.load('./selfie_avatar_32.npz')
    X_train = selfie_data['arr_0']
    
    # scale the data from the range [0, 255] to [-1, 1]
    # to work with the hyperbolic tangent
    X_train = (X_train.astype('float32') - 127.5) / 127.5
    
    return X_train

''' select real images '''
def generate_authentic_images(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    
    # generate authentic class labels (1)
    y = np.ones((n_samples, 1))
    
    # apply label smoothing for the range [0.7, 1.2]
    # to make a regularizing effect
    y = y - 0.3 + (np.random.random(y.shape) * 0.5)
    
    return X, y

''' generate points in latent space as input for the generator '''
def generate_latent_points(latent_dim, n_samples):
    z_noise = np.random.randn(latent_dim * n_samples)
    z_noise = z_noise.reshape(n_samples, latent_dim)
    
    return z_noise

''' use the generator to produce synthetic images, with class labels '''
def generate_synthetic_images(generator, latent_dim, n_samples):
    z_noise = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(z_noise)
    
    # create 'synthetic' class labels (0)
    y = np.zeros((n_samples, 1))
    
    return X, y

''' plot the images '''
def save_plot(examples, epoch, n=5):
    # rescale from the range [-1, 1] to [0, 1]
    examples = (examples+1) / 2.0
    
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(examples[i])
        
    plt.savefig('generated_plot_e%03d.png' % (epoch+1))

''' evaluate the discriminator, plot the images & save the model '''
def summarize_performance(epoch, g, d, dataset, latent_dim, n_samples=150):
    # prepare authentic samples
    X_authentic, y_authentic = generate_authentic_images(dataset, n_samples)
    _, accuracy_authentic = d.evaluate(X_authentic, y_authentic, verbose=0)
    
    # prepare synthetic images
    X_synthetic, y_synthetic = generate_synthetic_images(g, latent_dim, n_samples)
    _, accuracy_synthetic = d.evaluate(X_synthetic, y_synthetic, verbose=0)
    
    print('> Accuracy > authentic: %.0f%% || synthetic: %.0f%%' % (accuracy_authentic*100, accuracy_synthetic*100))
    save_plot(X_synthetic, epoch)
    g.save('generator_model_%03d.h5' % (epoch+1))

''' train the generator and discriminator models '''
def train(g, d, gan, dataset, latent_dim, epochs=3000, batch_size=8):
    batch_per_epoch = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_per_epoch / 2)
    
    for epoch in range(epochs):
        for batch in range(batch_per_epoch):
            # get random batch of authentic images
            # and update discriminator weights
            X_authentic, y_authentic = generate_authentic_images(dataset, half_batch)
            d_loss1, _ = d.train_on_batch(X_authentic, y_authentic)
            
            # get synthetic images
            # update discriminator weights
            X_synthetic, y_synthetic = generate_synthetic_images(g, latent_dim, half_batch)
            d_loss2, _ = d.train_on_batch(X_synthetic, y_synthetic)
            
            # get points from latent space to feed to the generator
            # and update the generator via the discriminator's error
            # and also set the labels to (1) to encourage the generator
            # to create more plausible images
            z_noise = generate_latent_points(latent_dim, batch_size)
            z_noise_labels =  np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(z_noise, z_noise_labels)
            
            print('>%d, %d/%d, D authentic loss: %.3f, D synthetic loss: %.3f, G loss: %.3f' % 
                 (epoch+1, batch+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
            
        # evaluate the model's performance
        if (epoch+1) % 200 == 0:
            summarize_performance(epoch, g, d, dataset, latent_dim)

In [ ]:
latent_dim = 100
g = build_generator(latent_dim)
d = build_discriminator(img_shape)
gan = build_gan(g, d)
dataset = load_real_samples()
train(g, d, gan, dataset, latent_dim)

In [ ]:
''' client to produce generator's images from latent space '''
import numpy as np
from keras.models import load_model

MODEL = './generator_model_600_og.h5'

z_noise = np.random.randn(100*25)
z_noise = z_noise.reshape(25, 100)

model = load_model(MODEL)
X = model.predict(z_noise)
X = (X+1) / 2.0

plt.imshow(X[4])

# show multiple designs
shoe_index = 0
fix, ax = plt.subplots(5, 5, figsize=(8, 6), sharey=False, sharex=False)
for i in range(5):
    for j in range(5):
        ax[i, j].imshow(X[shoe_index])
        ax[i, j].axis('off')
        shoe_index += 1
plt.tight_layout()